In [46]:
from pathlib import Path
import sys

here = Path.cwd()
repo_root = next(p for p in [here, *here.parents] if (p/"pyproject.toml").exists())
src = repo_root / "src"
if str(src) not in sys.path:
    sys.path.insert(0, str(src))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [47]:
from optimal_ipr.distributions import value_distribution, build_theta_distribution
from optimal_ipr.probability import build_subjective_probability
from optimal_ipr.cost import build_cost_function
from optimal_ipr.fee import build_fee_schedule
from optimal_ipr.lookup import build_lookup_tables
from optimal_ipr.outcomes import welfare_outcomes
from optimal_ipr.baseline import BaselineModel
from optimal_ipr.regulator import RegulatorModel
from optimal_ipr.government import GovernmentModel

In [48]:
# value distribution
v_grid, v_weights = value_distribution()

c:\Users\jesse\OneDrive\Research\Optimal IPR in a BEPS World\model\optimal-ipr\src\optimal_ipr\distributions\value_distribution.py:30: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  parsed = pd.to_datetime(raw, errors="coerce", infer_datetime_format=True)


In [49]:
# theta distribution
f, F, F_inv = build_theta_distribution(noise_level=0.20)

# scalar wrapper for F when a scalar is required
F_scalar = lambda t: float(F(np.array([t])))

In [50]:
p = build_subjective_probability(base_k=1, m_comp=0, F=F, F_inv=F_inv)

In [51]:
c = build_cost_function(f)

In [52]:
Z = build_fee_schedule(zeta=0.025)

In [53]:
tau_d_grid = np.array([0.15, 0.175, 0.20, 0.225, 0.25])
tau_f_grid = np.array([0.0, 0.05, 0.10])

# tau_d_grid = np.array([0.15, 0.20, 0.25, 0.30])
# tau_f_grid = np.array([0.0, 0.05, 0.10, 0.15])
bar_beta_grid = np.linspace(0.0, 1.0, 101)

reg_prefs = {
    "welfarist_balanced": {"phi": 0.5, "psi": 0.5},
    #"pro_innovation":     {"phi": 0.9, "psi": 0.1},
    #"public_good":        {"phi": 0.1, "psi": 0.9},
    #"utilitarian":        "special_case",
    #"rawlsian":           "special_case"
}

gov_prefs = {
    "utilitarian":                      lambda th: np.ones_like(th),                 
    "minimally_inequality_averse":      lambda th: 1.0 / (1.0 + th)**0.33,            
    "moderately_inequality_averse":     lambda th: 1.0 / (1.0 + th)**0.67,            
    "inequality_averse":                lambda th: 1.0 / (1.0 + th)**1.0,            
    "strongly_inequality_averse":       lambda th: 1.0 / (1.0 + th)**1.33,            
    "very_strongly_inequality_averse":  lambda th: 1.0 / (1.0 + th)**1.67,           
    "egalitarian":                      lambda th: 1.0 / (1.0 + th)**2.0,           
}

In [ ]:
theta_tilde_table, theta_winner_table, get_tau_d_index, get_tau_f_index, get_bar_beta_index, get_v_index = \
    build_lookup_tables(p, c, Z, f, F_scalar, tau_d_grid, tau_f_grid, bar_beta_grid, v_grid)

Processing tau pairs:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
results_table = welfare_outcomes(
    tau_d=tau_d_grid,
    tau_f=tau_f_grid,
    gov_prefs=gov_prefs,
    reg_prefs=reg_prefs,
    v_grid=v_grid,
    v_weights=v_weights,
    theta_tilde_table=theta_tilde_table,
    theta_winner_table=theta_winner_table,
    f=f,
    F=F,            # vectorized is fine; module wraps when needed
    F_inv=F_inv,
    p=p,
    c=c,
    Z=Z,
    feas=False,      # enforce feasibility
)

,Tau D,Tau F,Gov Pref,Reg Pref,Optimal Patent Breadth Cap,Expected Optimal Patent Breadth Granted,Welfare % Change,Innovator Welfare % Change,Imitator Welfare % Change,Non-Investor Welfare % Change
0,0.15,0.05,utilitarian,welfarist_balanced,1.00,1.00,0.000000,0.000000,0.000000,0.000000
1,0.15,0.05,elitist,welfarist_balanced,1.00,1.00,-0.000530,0.000000,-0.002614,0.000000
2,0.15,0.05,strongly_elitist,welfarist_balanced,1.00,1.00,-0.001056,0.000000,-0.005232,0.000000
3,0.15,0.05,egalitarian,welfarist_balanced,0.98,0.98,0.347592,-1.960014,9.693079,640.219109
4,0.15,0.05,strongly_egalitarian,welfarist_balanced,0.32,0.32,6.273651,-68.064947,218.425825,21273.160575
5,0.20,0.05,utilitarian,welfarist_balanced,0.98,0.98,0.017397,-1.957937,9.118747,555.651554
6,0.20,0.05,elitist,welfarist_balanced,1.00,1.00,-0.000489,0.000000,-0.002407,0.000000
7,0.20,0.05,strongly_elitist,welfarist_balanced,1.00,1.00,-0.000973,0.000000,-0.004819,0.000000
8,0.20,0.05,egalitarian,welfarist_balanced,0.88,0.88,0.766507,-12.093547,47.007555,3425.694510
9,0.20,0.05,strongly_egalitarian,welfarist_balanced,0.02,0.02,23.077444,-98.390804,285.938366,27549.861954


In [ ]:
gov_order = [
    "utilitarian",
    "minimally_inequality_averse",
    "moderately_inequality_averse",
    "inequality_averse",
    "strongly_inequality_averse",
    "very_strongly_inequality_averse",
    "egalitarian",
]

# convert column to categorical with explicit order
results_table["Gov Pref"] = pd.Categorical(
    results_table["Gov Pref"], categories=gov_order, ordered=True
)

# Example: sort ascending by Gov Pref, Reg Pref, Tau D, Tau F
results_table_sorted = results_table.sort_values(
    by=["Gov Pref", "Reg Pref", "Tau D", "Tau F"],
    ascending=[True, True, True, True]  # set to False if you need descending
).reset_index(drop=True)

results_table_sorted

In [ ]:
results_path = repo_root / "results"
if str(results_path) not in sys.path:
    sys.path.insert(0, str(results_path))

csv_path = results_path / "full_model_results.csv"
results_table.to_csv(csv_path, index=False)